In [1]:
import  torch
torch.cuda.device_count()

1

In [50]:
# %load finetune_textcnn.py
#!/usr/bin/env python
"""
FileName: finetune.py
Description:
Author: Stark Lv
Date: 2021/2/27 6:13 PM
Version: 0.1
"""

from transformers import get_linear_schedule_with_warmup
from TEXTCNN_pytorch import TEXTCNN
import torch
import logging
from torch import nn
from Data_generator_vocab import Corpus
import time
import datetime
from tqdm import tqdm
import numpy as np
import torch.nn.functional as F
import os
import json
import pandas as pd
from sklearn.metrics import roc_auc_score


def format_time(elapsed):
    elapsed_rounded = int(round(elapsed))
    return str(datetime.timedelta(seconds = elapsed_rounded))

class fintune():

    def __init__(self, model, corpus):

        self.set_device() #创建了self.device, self.n_gpu
        self.set_random_seed() #设置随机数种子,保证所有结果可以复现

        # 加载训练集、验证集、测试集
        print(f"{'加载训练集、验证集、测试集 Loading':*^80}")
        self.train_loader, self.valid_loader, self.test_loader = corpus.get_loaders()

        #加载model
        #self.model, self.tokenizer = self.load_model_tokenizer(model_name)


        self.model = model
        
        # move model to GPU
        if self.n_gpu > 1:
            device_ids = [0, 1, 2, 3]
            self.model =torch.nn.DataParallel(self.model, device_ids= device_ids)
        self.model = self.model.to(self.device)

        #loss function & optimizer
        # 带权重的损失函数, 若positive_label为1的f1_score为 0.75，positive_label为0的f1_score为0.85
        #self.loss_func = nn.CrossEntropyLoss(weight= torch.tensor([1.1765, 1.3333])).to(self.device)
        self.loss_func = nn.CrossEntropyLoss(weight=torch.tensor([0.5, 0.5])).to(self.device)
        self.optimizer = torch.optim.Adam(
                self.model.parameters(),
                lr = HYPERS["LR"],
                eps = 1e-8)




    """
    def load_model_tokenizer(self, model_name):
        #model_path = os.path.join(os.getcwd(), MODELS[model_name]["path"])
        model_path = MODELS[model_name]["path"]
        assert model_name in MODELS

        if self.retrain_model_path is None:

            print(f"### 从 {model_path} 加载预训练模型, 重新训练 ###")
            model = MODELS[model_name]["class"](model_path)
        else:
            print(f"{'加载已预训练好的模型,继续训练':*^80}")
            model = torch.load("./fintuned_model/" + self.retrain_model_path)

        print(f"## Model {self.model_name} loaded. ##")

        tokenizer = MODELS[model_name]["tokenizer"].from_pretrained(model_path)

        ## __class__.__name__输出类的名字
        print(f"## Tokenizer {tokenizer.__class__.__name__} loaded. ##")

        return model, tokenizer
    """
    
    def set_random_seed(self):
        # Set the seed value all over the place to make this reproducible.
        self.seed_val = 2021
        np.random.seed(self.seed_val)
        torch.manual_seed(self.seed_val)

    def set_device(self):
        ###设置日志时间的输出格式,说明在那段时间用过GPU
        LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
        DATE_FORMAT = "%m/%d/%Y %H:%M:%S %p"
        logging.basicConfig(filename='NLP_GPU.log', level=logging.DEBUG, format=LOG_FORMAT, datefmt=DATE_FORMAT)

        # 设置运行环境GPU
        self.device = torch.device("cuda:0")
        self.n_gpu = torch.cuda.device_count()
        logging.info(msg = f"\n Using GPU, {self.n_gpu} device available")

    def run_epoch(self):

        #list to store a number of quantities such as
        #training and validation loss, validation accuracy, and timings.
        training_stats = []


        #Total number of training steps is [number of batches × number of epochs]
        total_steps = len(self.train_loader) * HYPERS["EPOCHS"]
        # create the learning rate scheduler
        self.scheduler = get_linear_schedule_with_warmup(
            self.optimizer, num_warmup_steps = 100, num_training_steps = total_steps
        )

        total_t0 = time.time()

        print(f"### Trainng {HYPERS['EPOCHS']} EPOCHS start ###")
        for epoch in tqdm(range(1, HYPERS["EPOCHS"] + 1), unit = "epoch",
                          desc = f"Training All {HYPERS['EPOCHS']} Epochs"):
            # Measure how long the per training epoch takes
            t0 = time.time()

            # Put the model into training model
            print(f"### EPOCH {epoch} train  start")
            self.model.train()

            epoch_train_loss = self.train(epoch)
            # measure avg batch loss in specific epoch
            avg_train_loss = epoch_train_loss / len(self.train_loader)
            # Measure how long this epoch took.
            training_time = format_time(time.time() - t0)
            print(f"## per batch train loss for epoch {epoch} is {avg_train_loss} ",
                  f"total train time : {training_time}")

            t0 = time.time()

            # eval mode
            # 在eval模式下不会应用DROPOUT和BATCHNORM
            print(f"### EPOCH {epoch} val start")
            self.model.eval()

            epoch_eval_loss, eval_auc = self.eval(metric=roc_auc_score)
            avg_eval_loss = epoch_eval_loss / len(self.valid_loader)
            validation_time = format_time(time.time() - t0)
            print(f"## per batch valid loss for epoch {epoch} is {avg_eval_loss} ",
                  f"total valid time : {validation_time}, Validation auc score {eval_auc}")

            # Record all statistics from this epoch.
            training_stats.append(
                {
                    'Epoch': epoch,
                    'Train Loss': epoch_train_loss,
                    'Avg Train Loss': avg_train_loss,
                    'Train Time': training_time,
                    'Valid Loss': epoch_eval_loss,
                    'Avg Valid Loss': avg_eval_loss,
                    'Valid Time': validation_time,
                    'Valid AUC': eval_auc,
                    'Test Results Dir': 'Model{}_BATCH{}_LR{}/'.format(self.model_name,
                     HYPERS['BATCH_SIZE'], HYPERS['LR'])
                }
            )

        test_t0 = time.time()
        self.test()
        print("Test Time Consume {}".format(format_time(time.time() - test_t0)))
        self.save_stats(training_stats)
        print(f"{'Training complete !':*^80}")
        print(f"### Total Process took {format_time(time.time() - total_t0)} ####")


    def train(self, epoch):
        total_train_loss = 0

        for batch in tqdm(self.train_loader, desc=f"Epoch {epoch}>>>>train", unit="batch"):
            # input_ids, attention_mask, token_type_id和Label混在一起
            inputs = [data.to(self.device) for data in batch[:-1]]
            labels = batch[-1].to(self.device)
            outputs = self.model(*inputs)


            batch_loss = self.loss_func(outputs, labels)
            total_train_loss += batch_loss.to("cpu").data.numpy()
            # clear any previously calculated gradients before performing a backward pass.
            self.optimizer.zero_grad()

            # perform a backward pass to calculate the gradients
            batch_loss.backward()

            # normalization of the gradients to 1.0 to avoid exploding gradients
            nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)

            # update parameters and take a step using the computed gradient
            self.optimizer.step()

            #update the learning rate 
            self.scheduler.step()

            #print(f"per batch loss {batch_loss.to('cpu').data.numpy()}")

        return total_train_loss

    def eval(self, metric):
        total_eval_loss = 0
        predicted_scores = np.array([])
        target_labels = np.array([])
        #predicted_labels = np.array([])

        for batch in self.valid_loader:
            inputs = [data.to(self.device) for data in batch[:-1]]
            labels = batch[-1].to(self.device)

            # Tell pytorch not to bother with constructing the compute graph during
            # the forward pass, since this is only needed for backprop (training).
            with torch.no_grad():
                outputs = self.model(*inputs)
                loss = self.loss_func(outputs, labels)
                total_eval_loss += loss.to("cpu").data.numpy()

                # move logits and labels to CPU,存储在CPU上面的变量不能和存储在GPU上面的变量进行运算
                logits = F.softmax(outputs.to("cpu"), dim = 1).data.numpy()
                label_ids = labels.to("cpu").data.numpy()
                y_pred = logits[:, 1]
                #y_pred1 = np.argmax(logits, axis = 1)
                predicted_scores = np.append(predicted_scores, y_pred)
                #predicted_labels = np.append(predicted_labels, y_pred1)
                target_labels = np.append(target_labels, label_ids)


        task_auc = metric(target_labels, predicted_scores)

        #print(f"{'classification_report':*^80}")
        #print(classification_report(target_labels, predicted_labels))
        return total_eval_loss , task_auc

    def test(self):

        prediction_scores = np.array([])
        print(f"{'Predict task':*^80}")
        for batch in tqdm(self.test_loader, desc = "Predict Scores Loading", unit = "batch"):
            inputs = [data.to(self.device) for data in batch]

            with torch.no_grad():
                outputs = self.model(*inputs)
                logits = F.softmax(outputs.to("cpu"), dim=1).data.numpy()
                y_pred = logits[:, 1]
                prediction_scores = np.append(prediction_scores, y_pred)

        self.save_predictions(prediction_scores)

    def save_predictions(self, predicts):
        file_path = "./finetuned_results/" + 'Model{}_BATCH{}_EPOCH{}_LR{}/'.format(self.model_name,
                  HYPERS['BATCH_SIZE'],HYPERS['EPOCHS'], HYPERS['LR'])
        if not os.path.exists(file_path):
            os.mkdir(file_path)

        #输出当前时间
        dt = str(datetime.datetime.now()).split(" ")[0]
        filename = "{}_{}_(对应id和score).json".format(self.model_name, dt)
        with open(file_path + filename, "w") as fw:
            for idx, predict in enumerate(predicts):
                predict = {"id": idx, "label": predict}
                json.dump(predict, fw)
                if idx == len(predicts) - 1:
                    break
                fw.write("\n")

        filename2 = "{}_{}_result.tsv".format(self.model_name, dt)
        with open(file_path + filename2, "w") as fw:
            for idx, predict in enumerate(predicts):
                fw.write(str(predict))
                if idx == len(predicts) - 1:
                    break
                fw.write("\n")

    def save_stats(self ,stats):
        df = pd.DataFrame(stats)
        filename = 'Stats_{}_BATCH{}_Epoch{}_LR{}.csv'.format(self.model_name, HYPERS['BATCH_SIZE'],
                            HYPERS['EPOCHS'], HYPERS['LR'])
        df.to_csv("./finetuned_results/" + filename, sep = ",", encoding="utf-8", index = False)

    def save_model(self):
        # 保存训练好的模型
        model_name = 'Model{}_BATCH{}_Epoch{}_LR{}_TIME{}.pkl'.format(self.model_name, HYPERS['BATCH_SIZE'],
        HYPERS['EPOCHS'], HYPERS['LR'],time.strftime("%Y_%m_%d_%H_%M",time.localtime()))
        torch.save(self.model, "./finetuned_model/" + model_name)
        print("Fintuned model saved")


if __name__ == "__main__":
    ## 使用编号为4,5的两张显卡
    #os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    #os.environ["CUDA_VISIBLE_DEVICES"] = "4, 5, 6, 7"
    # global hyper parameters
    HYPERS = {
        "BATCH_SIZE":4,
        "LR": 5e-5,
        "EPOCHS": 7,
    }
    seed_val = 2021
    corpus = Corpus(HYPERS["BATCH_SIZE"], seed_val)
    class_num = 2
    maxlen = corpus.max_len
    max_features = len(corpus.vocab)
    embedding_dims = 300
    batch_size = 4
    dropout_rate = 0.2
    feature_size = 128
    kernel_sizes = [2, 3, 4, 5]

    print("Build model.....")
    model = TEXTCNN(maxlen=maxlen,
                    max_features=max_features,
                    embedding_dims=embedding_dims,
                    class_num=class_num,
                    kernel_sizes=kernel_sizes,
                    dropout_rate=dropout_rate,
                    feature_size=feature_size)
    corpus = Corpus(HYPERS["BATCH_SIZE"], seed_val)
    app = fintune(model, corpus)
    app.run_epoch()
    #app.save_model()

********************************Data_precessing*********************************
## dataset size is 121105
*******************************Load  dataset ....*******************************
## Load Datasets Consume 0:00:11 s ###
************************All Train and Test Data loaded !************************
Build model.....
********************************Data_precessing*********************************
## dataset size is 121105
*******************************Load  dataset ....*******************************
## Load Datasets Consume 0:00:09 s ###
************************All Train and Test Data loaded !************************


Epoch 1>>>>train:   0%|                                                                   | 0/27249 [00:00<?, ?batch/s]

*****************************加载训练集、验证集、测试集 Loading******************************
### Trainng 7 EPOCHS start ###
### EPOCH 1 train  start



Epoch 1>>>>train:   1%|▊                                                        | 393/27249 [00:16<18:20, 24.40batch/s]


Training All 7 Epochs:   0%|                                                                  | 0/7 [00:16<?, ?epoch/s]


KeyboardInterrupt: 

In [31]:
model

TEXTCNN(
  (embedding): Embedding(20604, 300)
  (relu): ReLU()
  (dropout): Dropout(p=0.2)
  (classifier): Linear(in_features=512, out_features=2, bias=True)
)

In [5]:
app.train_loader

In [6]:
model

TEXTCNN(
  (embedding): Embedding(20604, 300)
  (relu): ReLU()
  (dropout): Dropout(p=0.2)
  (classifier): Linear(in_features=512, out_features=2, bias=True)
)

In [9]:
for batch in app.train_loader:
    inputs = [data for data in batch[:-1]]
    labels = batch[-1]
    break

In [10]:
labels

tensor([0, 1, 1, 1])

In [30]:
inputs.dtype

torch.int64

In [44]:
from imp import reload
import TEXTCNN_pytorch
reload(TEXTCNN_pytorch)
from TEXTCNN_pytorch import TEXTCNN

In [45]:
class_num = 2
maxlen = corpus.max_len
max_features = len(corpus.vocab)
embedding_dims = 300
batch_size = 32
dropout_rate = 0.2
feature_size = 128
kernel_sizes = [2, 3, 4, 5]

print("Build model.....")
model = TEXTCNN(maxlen=maxlen,
                    max_features=max_features,
                    embedding_dims=embedding_dims,
                    class_num=class_num,
                    kernel_sizes=kernel_sizes,
                    dropout_rate=dropout_rate,
                    feature_size=feature_size)

Build model.....


In [4]:
inputs

NameError: name 'inputs' is not defined

In [47]:
model = model.to(app.device)

In [48]:
model

TEXTCNN(
  (embedding): Embedding(20604, 300)
  (conv1ds): ModuleList(
    (0): Conv1d(300, 128, kernel_size=(2,), stride=(1,))
    (1): Conv1d(300, 128, kernel_size=(3,), stride=(1,))
    (2): Conv1d(300, 128, kernel_size=(4,), stride=(1,))
    (3): Conv1d(300, 128, kernel_size=(5,), stride=(1,))
  )
  (relu): ReLU()
  (avgpools): ModuleList(
    (0): MaxPool1d(kernel_size=92, stride=92, padding=0, dilation=1, ceil_mode=False)
    (1): MaxPool1d(kernel_size=91, stride=91, padding=0, dilation=1, ceil_mode=False)
    (2): MaxPool1d(kernel_size=90, stride=90, padding=0, dilation=1, ceil_mode=False)
    (3): MaxPool1d(kernel_size=89, stride=89, padding=0, dilation=1, ceil_mode=False)
  )
  (dropout): Dropout(p=0.2)
  (classifier): Linear(in_features=512, out_features=2, bias=True)
)

In [38]:
inputs = inputs.to(app.device)

In [49]:
model(inputs)

tensor([[-0.5557,  0.0486],
        [-0.0607, -0.2917],
        [-0.4231, -0.6306],
        [-0.2986,  0.6190]], device='cuda:0', grad_fn=<AddmmBackward>)

In [40]:
inputs.device

device(type='cuda', index=0)

In [41]:
next(model.parameters()).device

device(type='cuda', index=0)

In [42]:
model

TEXTCNN(
  (embedding): Embedding(20604, 300)
  (relu): ReLU()
  (dropout): Dropout(p=0.2)
  (classifier): Linear(in_features=512, out_features=2, bias=True)
)

In [43]:
for param in model.parameters():
    print(type(param.data), param.size())


<class 'torch.Tensor'> torch.Size([20604, 300])
<class 'torch.Tensor'> torch.Size([2, 512])
<class 'torch.Tensor'> torch.Size([2])


In [51]:
import tarfile 
import os

tar = tarfile.open("textcnn.tar.gz", "w:gz")

for root, dir, files in os.walk("./code"):
    for file in files:
        fullpath = os.path.join(root, file)
        tar.add(fullpath)

tar.close()

In [2]:
m = 4
n = 3

In [3]:
[[1 for _ in range(m)] for _ in range(n)]

[[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]]

In [5]:
"@ . . .".split()

['@', '.', '.', '.']

In [6]:
import torch
inputs = torch.rand(2, 3, 5)

In [8]:
new_inputs = inputs.new()

In [9]:
inputs.new().resize_()

tensor([])

In [10]:
inputs.dim()

3

In [16]:
from itertools import repeat

In [14]:
list(itertools.repeat(1, 1))

[1]

In [21]:
inputs2 = inputs.new().resize_(inputs.size(0), *repeat(1, inputs.dim() - 2), inputs.size(2))

TypeError: resize_(): argument 'size' must be tuple of ints, but found element of type itertools.repeat at pos 2

In [27]:
inputs2.fill_(0)

tensor([[[0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.]]])

In [23]:
repeat(1, inputs.dim() - 2)

repeat(1, 1)

In [24]:
inputs.clone()

tensor([[[0.7501, 0.8988, 0.8986, 0.4217, 0.5892],
         [0.0373, 0.2572, 0.5352, 0.1509, 0.4278],
         [0.2676, 0.8277, 0.5438, 0.8423, 0.3216]],

        [[0.9667, 0.2692, 0.7169, 0.9250, 0.8498],
         [0.4199, 0.4934, 0.1143, 0.0123, 0.6788],
         [0.5399, 0.2404, 0.2149, 0.6032, 0.8619]]])

In [25]:
inputs

tensor([[[0.7501, 0.8988, 0.8986, 0.4217, 0.5892],
         [0.0373, 0.2572, 0.5352, 0.1509, 0.4278],
         [0.2676, 0.8277, 0.5438, 0.8423, 0.3216]],

        [[0.9667, 0.2692, 0.7169, 0.9250, 0.8498],
         [0.4199, 0.4934, 0.1143, 0.0123, 0.6788],
         [0.5399, 0.2404, 0.2149, 0.6032, 0.8619]]])

In [28]:
noise = inputs2.clone()

In [39]:
drop_prob = 0.2

In [40]:
noise.bernoulli_(1 - drop_prob).div_(1 - drop_prob)

tensor([[[1.2500, 1.2500, 1.2500, 1.2500, 0.0000]],

        [[1.2500, 1.2500, 1.2500, 1.2500, 1.2500]]])

In [41]:
noise

tensor([[[1.2500, 1.2500, 1.2500, 1.2500, 0.0000]],

        [[1.2500, 1.2500, 1.2500, 1.2500, 1.2500]]])

In [44]:
inputs.size()

torch.Size([2, 3, 5])

In [53]:
noise = noise.expand_as(inputs)

In [49]:
inputs3 = inputs.clone()

In [58]:
inputs3.mul_(noise)

tensor([[[0.9376, 1.1235, 1.1232, 0.5271, 0.0000],
         [0.0467, 0.3215, 0.6690, 0.1886, 0.0000],
         [0.3346, 1.0346, 0.6798, 1.0529, 0.0000]],

        [[1.2084, 0.3365, 0.8962, 1.1562, 1.0623],
         [0.5248, 0.6167, 0.1428, 0.0153, 0.8485],
         [0.6749, 0.3006, 0.2686, 0.7540, 1.0774]]])

In [60]:
inputs3

tensor([[[0.9376, 1.1235, 1.1232, 0.5271, 0.0000],
         [0.0467, 0.3215, 0.6690, 0.1886, 0.0000],
         [0.3346, 1.0346, 0.6798, 1.0529, 0.0000]],

        [[1.2084, 0.3365, 0.8962, 1.1562, 1.0623],
         [0.5248, 0.6167, 0.1428, 0.0153, 0.8485],
         [0.6749, 0.3006, 0.2686, 0.7540, 1.0774]]])

In [61]:
inputs

tensor([[[0.7501, 0.8988, 0.8986, 0.4217, 0.5892],
         [0.0373, 0.2572, 0.5352, 0.1509, 0.4278],
         [0.2676, 0.8277, 0.5438, 0.8423, 0.3216]],

        [[0.9667, 0.2692, 0.7169, 0.9250, 0.8498],
         [0.4199, 0.4934, 0.1143, 0.0123, 0.6788],
         [0.5399, 0.2404, 0.2149, 0.6032, 0.8619]]])

In [62]:
noise

tensor([[[1.2500, 1.2500, 1.2500, 1.2500, 0.0000],
         [1.2500, 1.2500, 1.2500, 1.2500, 0.0000],
         [1.2500, 1.2500, 1.2500, 1.2500, 0.0000]],

        [[1.2500, 1.2500, 1.2500, 1.2500, 1.2500],
         [1.2500, 1.2500, 1.2500, 1.2500, 1.2500],
         [1.2500, 1.2500, 1.2500, 1.2500, 1.2500]]])

In [1]:
from TEXTCNN_pytorch import TEXTCNN

ImportError: cannot import name 'Spatial_Dropout' from 'Spatial_Dropout' (E:\lv_python\NLP\阿里天池小布助手\TEXTCNN\code\Spatial_Dropout.py)

In [2]:
from Spatial_Dropout import SpatialDropout

In [ ]:
import torch

In [64]:
# %load finetune_textcnn.py
#!/usr/bin/env python
"""
FileName: finetune.py
Description:
Author: Stark Lv
Date: 2021/2/27 6:13 PM
Version: 0.1
"""

from transformers import get_linear_schedule_with_warmup
from TEXTCNN_pytorch import TEXTCNN
import torch
import logging
from torch import nn
from Data_generator_vocab import Corpus
import time
import datetime
from tqdm import tqdm
import numpy as np
import torch.nn.functional as F
import os
import json
import pandas as pd
from sklearn.metrics import roc_auc_score


def format_time(elapsed):
    elapsed_rounded = int(round(elapsed))
    return str(datetime.timedelta(seconds = elapsed_rounded))

class fintune():

    def __init__(self, model, corpus):

        self.set_device() #创建了self.device, self.n_gpu
        self.set_random_seed() #设置随机数种子,保证所有结果可以复现

        # 加载训练集、验证集、测试集
        print(f"{'加载训练集、验证集、测试集 Loading':*^80}")
        self.train_loader, self.valid_loader, self.test_loader = corpus.get_loaders()

        #加载model
        #self.model, self.tokenizer = self.load_model_tokenizer(model_name)


        self.model = model
        
        # move model to GPU
        if self.n_gpu > 1:
            device_ids = [0, 1, 2, 3]
            self.model =torch.nn.DataParallel(self.model, device_ids= device_ids)
        self.model = self.model.to(self.device)

        #loss function & optimizer
        # 带权重的损失函数, 若positive_label为1的f1_score为 0.75，positive_label为0的f1_score为0.85
        #self.loss_func = nn.CrossEntropyLoss(weight= torch.tensor([1.1765, 1.3333])).to(self.device)
        self.loss_func = nn.CrossEntropyLoss(weight=torch.tensor([0.5, 0.5])).to(self.device)
        self.optimizer = torch.optim.Adam(
                self.model.parameters(),
                lr = HYPERS["LR"],
                eps = 1e-8)




    """
    def load_model_tokenizer(self, model_name):
        #model_path = os.path.join(os.getcwd(), MODELS[model_name]["path"])
        model_path = MODELS[model_name]["path"]
        assert model_name in MODELS

        if self.retrain_model_path is None:

            print(f"### 从 {model_path} 加载预训练模型, 重新训练 ###")
            model = MODELS[model_name]["class"](model_path)
        else:
            print(f"{'加载已预训练好的模型,继续训练':*^80}")
            model = torch.load("./fintuned_model/" + self.retrain_model_path)

        print(f"## Model {self.model_name} loaded. ##")

        tokenizer = MODELS[model_name]["tokenizer"].from_pretrained(model_path)

        ## __class__.__name__输出类的名字
        print(f"## Tokenizer {tokenizer.__class__.__name__} loaded. ##")

        return model, tokenizer
    """
    
    def set_random_seed(self):
        # Set the seed value all over the place to make this reproducible.
        self.seed_val = 2021
        np.random.seed(self.seed_val)
        torch.manual_seed(self.seed_val)

    def set_device(self):
        ###设置日志时间的输出格式,说明在那段时间用过GPU
        LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
        DATE_FORMAT = "%m/%d/%Y %H:%M:%S %p"
        logging.basicConfig(filename='NLP_GPU.log', level=logging.DEBUG, format=LOG_FORMAT, datefmt=DATE_FORMAT)

        # 设置运行环境GPU
        self.device = torch.device("cuda:0")
        self.n_gpu = torch.cuda.device_count()
        logging.info(msg = f"\n Using GPU, {self.n_gpu} device available")

    def run_epoch(self):

        #list to store a number of quantities such as
        #training and validation loss, validation accuracy, and timings.
        training_stats = []


        #Total number of training steps is [number of batches × number of epochs]
        total_steps = len(self.train_loader) * HYPERS["EPOCHS"]
        # create the learning rate scheduler
        self.scheduler = get_linear_schedule_with_warmup(
            self.optimizer, num_warmup_steps = 100, num_training_steps = total_steps
        )

        total_t0 = time.time()

        print(f"### Trainng {HYPERS['EPOCHS']} EPOCHS start ###")
        for epoch in tqdm(range(1, HYPERS["EPOCHS"] + 1), unit = "epoch",
                          desc = f"Training All {HYPERS['EPOCHS']} Epochs"):
            # Measure how long the per training epoch takes
            t0 = time.time()

            # Put the model into training model
            print(f"### EPOCH {epoch} train  start")
            self.model.train()

            epoch_train_loss = self.train(epoch)
            # measure avg batch loss in specific epoch
            avg_train_loss = epoch_train_loss / len(self.train_loader)
            # Measure how long this epoch took.
            training_time = format_time(time.time() - t0)
            print(f"## per batch train loss for epoch {epoch} is {avg_train_loss} ",
                  f"total train time : {training_time}")

            t0 = time.time()

            # eval mode
            # 在eval模式下不会应用DROPOUT和BATCHNORM
            print(f"### EPOCH {epoch} val start")
            self.model.eval()

            epoch_eval_loss, eval_auc = self.eval(metric=roc_auc_score)
            avg_eval_loss = epoch_eval_loss / len(self.valid_loader)
            validation_time = format_time(time.time() - t0)
            print(f"## per batch valid loss for epoch {epoch} is {avg_eval_loss} ",
                  f"total valid time : {validation_time}, Validation auc score {eval_auc}")

            # Record all statistics from this epoch.
            training_stats.append(
                {
                    'Epoch': epoch,
                    'Train Loss': epoch_train_loss,
                    'Avg Train Loss': avg_train_loss,
                    'Train Time': training_time,
                    'Valid Loss': epoch_eval_loss,
                    'Avg Valid Loss': avg_eval_loss,
                    'Valid Time': validation_time,
                    'Valid AUC': eval_auc,
                    'Test Results Dir': 'Model{}_BATCH{}_LR{}/'.format(self.model_name,
                     HYPERS['BATCH_SIZE'], HYPERS['LR'])
                }
            )

        test_t0 = time.time()
        self.test()
        print("Test Time Consume {}".format(format_time(time.time() - test_t0)))
        self.save_stats(training_stats)
        print(f"{'Training complete !':*^80}")
        print(f"### Total Process took {format_time(time.time() - total_t0)} ####")


    def train(self, epoch):
        total_train_loss = 0

        for batch in tqdm(self.train_loader, desc=f"Epoch {epoch}>>>>train", unit="batch"):
            # input_ids, attention_mask, token_type_id和Label混在一起
            inputs = [data.to(self.device) for data in batch[:-1]]
            labels = batch[-1].to(self.device)
            outputs = self.model(*inputs)


            batch_loss = self.loss_func(outputs, labels)
            total_train_loss += batch_loss.to("cpu").data.numpy()
            # clear any previously calculated gradients before performing a backward pass.
            self.optimizer.zero_grad()

            # perform a backward pass to calculate the gradients
            batch_loss.backward()

            # normalization of the gradients to 1.0 to avoid exploding gradients
            nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)

            # update parameters and take a step using the computed gradient
            self.optimizer.step()

            #update the learning rate 
            self.scheduler.step()

            #print(f"per batch loss {batch_loss.to('cpu').data.numpy()}")

        return total_train_loss

    def eval(self, metric):
        total_eval_loss = 0
        predicted_scores = np.array([])
        target_labels = np.array([])
        #predicted_labels = np.array([])

        for batch in self.valid_loader:
            inputs = [data.to(self.device) for data in batch[:-1]]
            labels = batch[-1].to(self.device)

            # Tell pytorch not to bother with constructing the compute graph during
            # the forward pass, since this is only needed for backprop (training).
            with torch.no_grad():
                outputs = self.model(*inputs)
                loss = self.loss_func(outputs, labels)
                total_eval_loss += loss.to("cpu").data.numpy()

                # move logits and labels to CPU,存储在CPU上面的变量不能和存储在GPU上面的变量进行运算
                logits = F.softmax(outputs.to("cpu"), dim = 1).data.numpy()
                label_ids = labels.to("cpu").data.numpy()
                y_pred = logits[:, 1]
                #y_pred1 = np.argmax(logits, axis = 1)
                predicted_scores = np.append(predicted_scores, y_pred)
                #predicted_labels = np.append(predicted_labels, y_pred1)
                target_labels = np.append(target_labels, label_ids)


        task_auc = metric(target_labels, predicted_scores)

        #print(f"{'classification_report':*^80}")
        #print(classification_report(target_labels, predicted_labels))
        return total_eval_loss , task_auc

    def test(self):

        prediction_scores = np.array([])
        print(f"{'Predict task':*^80}")
        for batch in tqdm(self.test_loader, desc = "Predict Scores Loading", unit = "batch"):
            inputs = [data.to(self.device) for data in batch]

            with torch.no_grad():
                outputs = self.model(*inputs)
                logits = F.softmax(outputs.to("cpu"), dim=1).data.numpy()
                y_pred = logits[:, 1]
                prediction_scores = np.append(prediction_scores, y_pred)

        self.save_predictions(prediction_scores)

    def save_predictions(self, predicts):
        file_path = "./finetuned_results/" + 'Model{}_BATCH{}_EPOCH{}_LR{}/'.format(self.model_name,
                  HYPERS['BATCH_SIZE'],HYPERS['EPOCHS'], HYPERS['LR'])
        if not os.path.exists(file_path):
            os.mkdir(file_path)

        #输出当前时间
        dt = str(datetime.datetime.now()).split(" ")[0]
        filename = "{}_{}_(对应id和score).json".format(self.model_name, dt)
        with open(file_path + filename, "w") as fw:
            for idx, predict in enumerate(predicts):
                predict = {"id": idx, "label": predict}
                json.dump(predict, fw)
                if idx == len(predicts) - 1:
                    break
                fw.write("\n")

        filename2 = "{}_{}_result.tsv".format(self.model_name, dt)
        with open(file_path + filename2, "w") as fw:
            for idx, predict in enumerate(predicts):
                fw.write(str(predict))
                if idx == len(predicts) - 1:
                    break
                fw.write("\n")

    def save_stats(self ,stats):
        df = pd.DataFrame(stats)
        filename = 'Stats_{}_BATCH{}_Epoch{}_LR{}.csv'.format(self.model_name, HYPERS['BATCH_SIZE'],
                            HYPERS['EPOCHS'], HYPERS['LR'])
        df.to_csv("./finetuned_results/" + filename, sep = ",", encoding="utf-8", index = False)

    def save_model(self):
        # 保存训练好的模型
        model_name = 'Model{}_BATCH{}_Epoch{}_LR{}_TIME{}.pkl'.format(self.model_name, HYPERS['BATCH_SIZE'],
        HYPERS['EPOCHS'], HYPERS['LR'],time.strftime("%Y_%m_%d_%H_%M",time.localtime()))
        torch.save(self.model, "./finetuned_model/" + model_name)
        print("Fintuned model saved")


if __name__ == "__main__":
    ## 使用编号为4,5的两张显卡
    #os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    #os.environ["CUDA_VISIBLE_DEVICES"] = "4, 5, 6, 7"
    # global hyper parameters
    HYPERS = {
        "BATCH_SIZE":32 * 4,
        "LR": 5e-5,
        "EPOCHS": 7,
    }
    seed_val = 2021
    corpus = Corpus(HYPERS["BATCH_SIZE"], seed_val)
    class_num = 2
    maxlen = corpus.max_len
    max_features = len(corpus.vocab)
    embedding_dims = 400
    batch_size = 4
    dropout_rate = 0.2
    feature_size = 1024
    kernel_sizes = [2, 3, 4, 5]

    print("Build model.....")
    model = TEXTCNN(maxlen=maxlen,
                    max_features=max_features,
                    embedding_dims=embedding_dims,
                    class_num=class_num,
                    kernel_sizes=kernel_sizes,
                    dropout_rate=dropout_rate,
                    feature_size=feature_size)
    corpus = Corpus(HYPERS["BATCH_SIZE"], seed_val)
    app = fintune(model, corpus)
    #app.run_epoch()
    #app.save_model()

********************************Data_precessing*********************************
## dataset size is 121105
*******************************Load  dataset ....*******************************
## Load Datasets Consume 0:00:07 s ###
************************All Train and Test Data loaded !************************
Build model.....
********************************Data_precessing*********************************
## dataset size is 121105
*******************************Load  dataset ....*******************************
## Load Datasets Consume 0:00:07 s ###
************************All Train and Test Data loaded !************************
*****************************加载训练集、验证集、测试集 Loading******************************


In [90]:
for batch in app.train_loader:
    inputs = batch[:-1]
    labels = batch[-1]
    break

In [135]:

import TEXTCNN_pytorch
reload(TEXTCNN_pytorch)
from TEXTCNN_pytorch import TEXTCNN

In [91]:
inputs = [inputs[0][:4]]

In [136]:
model = TEXTCNN(maxlen=maxlen,
                    max_features=max_features,
                    embedding_dims=embedding_dims,
                    class_num=class_num,
                    kernel_sizes=kernel_sizes,
                    dropout_rate=dropout_rate,
                    feature_size=feature_size)

In [137]:
model(*inputs)

tensor([[0.5152, 0.4848],
        [0.4941, 0.5059],
        [0.5498, 0.4502],
        [0.5403, 0.4597]], grad_fn=<SoftmaxBackward>)

In [134]:
inputs

[tensor([[    2,   152,  3502,     9,   149, 10705,    54,   716,    28,   109,
            110,    72,     3,   947,  2257,   760,   906, 14553,    58,    93,
            110,     3,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0],
         [    2,    10,     7,    48,    21,     3,    23,   236,   198,     3,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
        

In [130]:
inputs[0].size()

torch.Size([4, 93])

In [140]:
x = [torch.rand(2, 3)]
def change(x):
    print(x)
    print(type(x))
    return x

In [141]:
change(*x)

tensor([[0.9661, 0.0943, 0.9204],
        [0.5622, 0.1390, 0.7411]])
<class 'torch.Tensor'>


tensor([[0.9661, 0.0943, 0.9204],
        [0.5622, 0.1390, 0.7411]])

In [2]:
# %load finetune_textcnn.py
#!/usr/bin/env python
"""
FileName: finetune.py
Description:
Author: Stark Lv
Date: 2021/2/27 6:13 PM
Version: 0.1
"""

from transformers import get_linear_schedule_with_warmup
from TEXTCNN_pytorch import TEXTCNN
import torch
import logging
from torch import nn
from Data_generator_vocab import Corpus
import time
import datetime
from tqdm import tqdm
import numpy as np
import torch.nn.functional as F
import os
import json
import pandas as pd
from sklearn.metrics import roc_auc_score


def format_time(elapsed):
    elapsed_rounded = int(round(elapsed))
    return str(datetime.timedelta(seconds = elapsed_rounded))

class fintune():

    def __init__(self, model, corpus):

        self.set_device() #创建了self.device, self.n_gpu
        self.set_random_seed() #设置随机数种子,保证所有结果可以复现

        # 加载训练集、验证集、测试集
        print(f"{'加载训练集、验证集、测试集 Loading':*^80}")
        self.train_loader, self.valid_loader, self.test_loader = corpus.get_loaders()

        #加载model
        #self.model, self.tokenizer = self.load_model_tokenizer(model_name)


        self.model = model
        
        # move model to GPU
        if self.n_gpu > 1:
            device_ids = [0, 1, 2, 3]
            self.model =torch.nn.DataParallel(self.model, device_ids= device_ids)
        self.model = self.model.to(self.device)

        #loss function & optimizer
        # 带权重的损失函数, 若positive_label为1的f1_score为 0.75，positive_label为0的f1_score为0.85
        #self.loss_func = nn.CrossEntropyLoss(weight= torch.tensor([1.1765, 1.3333])).to(self.device)
        self.loss_func = nn.CrossEntropyLoss(weight=torch.tensor([0.5, 0.5])).to(self.device)
        self.optimizer = torch.optim.Adam(
                self.model.parameters(),
                lr = HYPERS["LR"],
                eps = 1e-8)




    """
    def load_model_tokenizer(self, model_name):
        #model_path = os.path.join(os.getcwd(), MODELS[model_name]["path"])
        model_path = MODELS[model_name]["path"]
        assert model_name in MODELS

        if self.retrain_model_path is None:

            print(f"### 从 {model_path} 加载预训练模型, 重新训练 ###")
            model = MODELS[model_name]["class"](model_path)
        else:
            print(f"{'加载已预训练好的模型,继续训练':*^80}")
            model = torch.load("./fintuned_model/" + self.retrain_model_path)

        print(f"## Model {self.model_name} loaded. ##")

        tokenizer = MODELS[model_name]["tokenizer"].from_pretrained(model_path)

        ## __class__.__name__输出类的名字
        print(f"## Tokenizer {tokenizer.__class__.__name__} loaded. ##")

        return model, tokenizer
    """
    
    def set_random_seed(self):
        # Set the seed value all over the place to make this reproducible.
        self.seed_val = 2021
        np.random.seed(self.seed_val)
        torch.manual_seed(self.seed_val)

    def set_device(self):
        ###设置日志时间的输出格式,说明在那段时间用过GPU
        LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
        DATE_FORMAT = "%m/%d/%Y %H:%M:%S %p"
        logging.basicConfig(filename='NLP_GPU.log', level=logging.DEBUG, format=LOG_FORMAT, datefmt=DATE_FORMAT)

        # 设置运行环境GPU
        self.device = torch.device("cuda:0")
        self.n_gpu = torch.cuda.device_count()
        logging.info(msg = f"\n Using GPU, {self.n_gpu} device available")

    def run_epoch(self):

        #list to store a number of quantities such as
        #training and validation loss, validation accuracy, and timings.
        training_stats = []


        #Total number of training steps is [number of batches × number of epochs]
        total_steps = len(self.train_loader) * HYPERS["EPOCHS"]
        # create the learning rate scheduler
        self.scheduler = get_linear_schedule_with_warmup(
            self.optimizer, num_warmup_steps = 100, num_training_steps = total_steps
        )

        total_t0 = time.time()

        print(f"### Trainng {HYPERS['EPOCHS']} EPOCHS start ###")
        for epoch in tqdm(range(1, HYPERS["EPOCHS"] + 1), unit = "epoch",
                          desc = f"Training All {HYPERS['EPOCHS']} Epochs"):
            # Measure how long the per training epoch takes
            t0 = time.time()

            # Put the model into training model
            print(f"### EPOCH {epoch} train  start")
            self.model.train()

            epoch_train_loss = self.train(epoch)
            # measure avg batch loss in specific epoch
            avg_train_loss = epoch_train_loss / len(self.train_loader)
            # Measure how long this epoch took.
            training_time = format_time(time.time() - t0)
            print(f"## per batch train loss for epoch {epoch} is {avg_train_loss} ",
                  f"total train time : {training_time}")

            t0 = time.time()

            # eval mode
            # 在eval模式下不会应用DROPOUT和BATCHNORM
            print(f"### EPOCH {epoch} val start")
            self.model.eval()

            epoch_eval_loss, eval_auc = self.eval(metric=roc_auc_score)
            avg_eval_loss = epoch_eval_loss / len(self.valid_loader)
            validation_time = format_time(time.time() - t0)
            print(f"## per batch valid loss for epoch {epoch} is {avg_eval_loss} ",
                  f"total valid time : {validation_time}, Validation auc score {eval_auc}")

            # Record all statistics from this epoch.
            training_stats.append(
                {
                    'Epoch': epoch,
                    'Train Loss': epoch_train_loss,
                    'Avg Train Loss': avg_train_loss,
                    'Train Time': training_time,
                    'Valid Loss': epoch_eval_loss,
                    'Avg Valid Loss': avg_eval_loss,
                    'Valid Time': validation_time,
                    'Valid AUC': eval_auc,
                    'Test Results Dir': 'Model{}_BATCH{}_LR{}/'.format(self.model_name,
                     HYPERS['BATCH_SIZE'], HYPERS['LR'])
                }
            )

        test_t0 = time.time()
        self.test()
        print("Test Time Consume {}".format(format_time(time.time() - test_t0)))
        self.save_stats(training_stats)
        print(f"{'Training complete !':*^80}")
        print(f"### Total Process took {format_time(time.time() - total_t0)} ####")


    def train(self, epoch):
        total_train_loss = 0

        for batch in tqdm(self.train_loader, desc=f"Epoch {epoch}>>>>train", unit="batch"):
            # input_ids, attention_mask, token_type_id和Label混在一起
            inputs = [data.to(self.device) for data in batch[:-1]]
            labels = batch[-1].to(self.device)
            outputs = self.model(*inputs)


            batch_loss = self.loss_func(outputs, labels)
            total_train_loss += batch_loss.to("cpu").data.numpy()
            # clear any previously calculated gradients before performing a backward pass.
            self.optimizer.zero_grad()

            # perform a backward pass to calculate the gradients
            batch_loss.backward()

            # normalization of the gradients to 1.0 to avoid exploding gradients
            nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)

            # update parameters and take a step using the computed gradient
            self.optimizer.step()

            #update the learning rate 
            self.scheduler.step()

            #print(f"per batch loss {batch_loss.to('cpu').data.numpy()}")

        return total_train_loss

    def eval(self, metric):
        total_eval_loss = 0
        predicted_scores = np.array([])
        target_labels = np.array([])
        #predicted_labels = np.array([])

        for batch in self.valid_loader:
            inputs = [data.to(self.device) for data in batch[:-1]]
            labels = batch[-1].to(self.device)

            # Tell pytorch not to bother with constructing the compute graph during
            # the forward pass, since this is only needed for backprop (training).
            with torch.no_grad():
                outputs = self.model(*inputs)
                loss = self.loss_func(outputs, labels)
                total_eval_loss += loss.to("cpu").data.numpy()

                # move logits and labels to CPU,存储在CPU上面的变量不能和存储在GPU上面的变量进行运算
                logits = F.softmax(outputs.to("cpu"), dim = 1).data.numpy()
                label_ids = labels.to("cpu").data.numpy()
                y_pred = logits[:, 1]
                #y_pred1 = np.argmax(logits, axis = 1)
                predicted_scores = np.append(predicted_scores, y_pred)
                #predicted_labels = np.append(predicted_labels, y_pred1)
                target_labels = np.append(target_labels, label_ids)


        task_auc = metric(target_labels, predicted_scores)

        #print(f"{'classification_report':*^80}")
        #print(classification_report(target_labels, predicted_labels))
        return total_eval_loss , task_auc

    def test(self):

        prediction_scores = np.array([])
        print(f"{'Predict task':*^80}")
        for batch in tqdm(self.test_loader, desc = "Predict Scores Loading", unit = "batch"):
            inputs = [data.to(self.device) for data in batch][0]

            with torch.no_grad():
                outputs = self.model(inputs)
                logits = F.softmax(outputs.to("cpu"), dim=1).data.numpy()
                y_pred = logits[:, 1]
                prediction_scores = np.append(prediction_scores, y_pred)

        self.save_predictions(prediction_scores)

    def save_predictions(self, predicts):
        file_path = "./finetuned_results/" + 'Model{}_BATCH{}_EPOCH{}_LR{}/'.format(self.model_name,
                  HYPERS['BATCH_SIZE'],HYPERS['EPOCHS'], HYPERS['LR'])
        if not os.path.exists(file_path):
            os.mkdir(file_path)

        #输出当前时间
        dt = str(datetime.datetime.now()).split(" ")[0]
        filename = "{}_{}_(对应id和score).json".format(self.model_name, dt)
        with open(file_path + filename, "w") as fw:
            for idx, predict in enumerate(predicts):
                predict = {"id": idx, "label": predict}
                json.dump(predict, fw)
                if idx == len(predicts) - 1:
                    break
                fw.write("\n")

        filename2 = "{}_{}_result.tsv".format(self.model_name, dt)
        with open(file_path + filename2, "w") as fw:
            for idx, predict in enumerate(predicts):
                fw.write(str(predict))
                if idx == len(predicts) - 1:
                    break
                fw.write("\n")

    def save_stats(self ,stats):
        df = pd.DataFrame(stats)
        filename = 'Stats_{}_BATCH{}_Epoch{}_LR{}.csv'.format(self.model_name, HYPERS['BATCH_SIZE'],
                            HYPERS['EPOCHS'], HYPERS['LR'])
        df.to_csv("./finetuned_results/" + filename, sep = ",", encoding="utf-8", index = False)

    def save_model(self):
        # 保存训练好的模型
        model_name = 'Model{}_BATCH{}_Epoch{}_LR{}_TIME{}.pkl'.format(self.model_name, HYPERS['BATCH_SIZE'],
        HYPERS['EPOCHS'], HYPERS['LR'],time.strftime("%Y_%m_%d_%H_%M",time.localtime()))
        torch.save(self.model, "./finetuned_model/" + model_name)
        print("Fintuned model saved")


if __name__ == "__main__":
    ## 使用编号为4,5的两张显卡
    #os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    #os.environ["CUDA_VISIBLE_DEVICES"] = "4, 5, 6, 7"
    # global hyper parameters
    HYPERS = {
        "BATCH_SIZE":4,
        "LR": 5e-5,
        "EPOCHS": 7,
    }
    seed_val = 2021
    corpus = Corpus(HYPERS["BATCH_SIZE"], seed_val)
    class_num = 2
    maxlen = corpus.max_len
    max_features = len(corpus.vocab)
    embedding_dims = 400
    dropout_rate = 0.2
    feature_size = 1024
    kernel_sizes = [2, 3, 4, 5]

    print("Build model.....")
    model = TEXTCNN(maxlen=maxlen,
                    max_features=max_features,
                    embedding_dims=embedding_dims,
                    class_num=class_num,
                    kernel_sizes=kernel_sizes,
                    dropout_rate=dropout_rate,
                    feature_size=feature_size)
    app = fintune(model, corpus)
    app.run_epoch()
    #app.save_model()

********************************Data_precessing*********************************
## dataset size is 121105
*******************************Load  dataset ....*******************************
## Load Datasets Consume 0:00:06 s ###
************************All Train and Test Data loaded !************************
Build model.....
*****************************加载训练集、验证集、测试集 Loading******************************


Epoch 1>>>>train:   0%|                                                                   | 0/27249 [00:00<?, ?batch/s]

### Trainng 7 EPOCHS start ###
### EPOCH 1 train  start



Training All 7 Epochs:   0%|                                                                  | 0/7 [00:10<?, ?epoch/s]


KeyboardInterrupt: 

In [18]:
import pandas as pd
data = pd.read_csv("E:/lv_python/NLP/阿里天池小布助手/finetuned_results/2021-03-07_Linear_result.tsv", header = None, names = ["score"])
data1 = pd.read_csv(r"E:\lv_python\NLP\阿里天池小布助手\finetuned_results\TextCNN_2021-03-11_result.tsv", header = None, names = ["score"])

In [19]:
data

,score
0,0.000220
1,0.969897
2,0.006519
3,0.056620
4,0.029994
...,...
24995,0.000209
24996,0.000881
24997,0.996679
24998,0.003614


In [20]:
data1

,score
0,0.268941
1,0.722515
2,0.268943
3,0.269112
4,0.276338
...,...
24995,0.270007
24996,0.271409
24997,0.731059
24998,0.268964


In [26]:
for idx, (x, y) in enumerate(zip(data["score"], data1["score"])):
    print(idx, x, y)

0 0.00022005951905157417 0.2689414322376251
1 0.9698974490165709 0.7225145697593689
2 0.0065185949206352225 0.26894333958625793
3 0.05662039294838905 0.26911163330078125
4 0.029993537813425064 0.2763379514217377
5 0.9974032044410706 0.7310423851013184
6 0.00017127387400250882 0.2689414918422699
7 0.9992104768753052 0.731052041053772
8 0.7372874021530151 0.2755856812000275
9 0.9565937519073486 0.7209820747375488
10 0.9158976674079896 0.7179679274559021
11 9.49774621403776e-05 0.2689414620399475
12 0.9853309392929076 0.7310585975646973
13 6.0257705627009266e-05 0.2689415216445923
14 0.01780320517718792 0.2760138809680939
15 0.7582051753997803 0.2690070569515228
16 0.0023349737748503685 0.2689602673053741
17 0.0016782927559688687 0.2690020203590393
18 0.032146137207746506 0.27056971192359924
19 0.0019852144178003073 0.2701297700405121
20 0.4039240777492523 0.270845890045166
21 0.9985894560813904 0.7197993993759155
22 0.000705293205101043 0.2689414322376251
23 0.9991905093193054 0.73105859

763 0.07398803532123566 0.2689414322376251
764 0.008337900042533875 0.2689417004585266
765 0.9965448975563048 0.2689512073993683
766 0.11338501423597334 0.2692548036575317
767 0.003106718650087714 0.2689414322376251
768 0.9658424258232116 0.7308250069618225
769 0.9840934276580809 0.7310066819190979
770 0.08531506359577179 0.27135369181633
771 0.9952730536460876 0.7309961915016174
772 0.00021343711705412718 0.2689489722251892
773 0.9630703926086426 0.7310551404953003
774 0.00014220982848200947 0.2698700428009033
775 0.00012957953731529417 0.26894161105155945
776 0.9930949211120604 0.6717510819435121
777 0.9992353916168212 0.7310572862625122
778 0.004303284455090761 0.2689806818962097
779 0.9914680719375609 0.7300203442573547
780 0.0001837288436945528 0.2689645290374756
781 0.03561822697520256 0.2690106332302093
782 0.9874548912048341 0.7142139077186584
783 0.9998443126678468 0.7309423685073853
784 0.991567313671112 0.7306256890296936
785 0.07574532926082611 0.26916754245758057
786 0.993

1596 0.8104285597801208 0.5609470009803772
1597 7.391176040982829e-05 0.2691025733947754
1598 0.00023812337894923985 0.2689414322376251
1599 0.9978037476539612 0.7310564517974854
1600 0.996138870716095 0.7309667468070984
1601 0.014375693164765838 0.27243223786354065
1602 5.831550879520364e-05 0.2690715491771698
1603 0.9966975450515748 0.7204868793487549
1604 0.9860952496528624 0.27244627475738525
1605 0.9485780596733092 0.7284643650054932
1606 0.07043027877807617 0.27056294679641724
1607 0.0006488646613433957 0.2727612853050232
1608 0.9436883330345154 0.2689414322376251
1609 0.004615960642695427 0.2689426839351654
1610 0.973170042037964 0.7310161590576172
1611 0.0001378755841869861 0.2689414620399475
1612 0.989285171031952 0.7259142398834229
1613 0.009367463178932669 0.2689569890499115
1614 0.9807820320129396 0.7310574650764465
1615 0.9495787620544434 0.7310292720794678
1616 0.002622194122523069 0.2689414322376251
1617 0.19605199992656708 0.4977252185344696
1618 0.9938993453979492 0.73

2429 0.9967544674873352 0.7310584187507629
2430 0.5951200723648071 0.26896539330482483
2431 0.9918877482414246 0.7310238480567932
2432 0.01583172380924225 0.6136884689331055
2433 0.009356982074677944 0.2696368992328644
2434 0.9875475168228148 0.7302687168121338
2435 0.988788902759552 0.7305084466934204
2436 0.00012087672803318128 0.2689415216445923
2437 0.904111623764038 0.7309046387672424
2438 0.8034133911132812 0.7310529947280884
2439 0.0003213121672160924 0.2689414322376251
2440 0.9966468214988708 0.7310569286346436
2441 0.2557486593723297 0.4181042015552521
2442 0.004692669026553631 0.2689428627490997
2443 0.9980731010437012 0.7309066653251648
2444 0.06345824152231216 0.7308764457702637
2445 0.8817205429077148 0.6431431770324707
2446 0.9877380132675172 0.7310540676116943
2447 0.99686461687088 0.731053352355957
2448 0.963754415512085 0.4422342479228973
2449 0.9959557056427002 0.7310444712638855
2450 0.9924513697624208 0.7310193181037903
2451 0.3587362468242645 0.6698252558708191
245

3262 0.5171074271202087 0.388226717710495
3263 0.9994561076164246 0.7310582995414734
3264 0.9959914088249208 0.7127578854560852
3265 0.0036938393022865053 0.2689414322376251
3266 0.9951753616333008 0.7310585975646973
3267 7.645636651432142e-05 0.2689532935619354
3268 0.9967670440673828 0.7310360074043274
3269 0.00011342218931531534 0.2695719301700592
3270 0.3855124413967133 0.7301498055458069
3271 0.0003943385672755539 0.2689414322376251
3272 0.005746967624872923 0.2689414322376251
3273 0.0007943408563733102 0.26902976632118225
3274 0.005489120725542307 0.2720305621623993
3275 0.999543070793152 0.7310583591461182
3276 0.009399895556271076 0.2689415514469147
3277 0.9629215002059937 0.7309508323669434
3278 0.00039132824167609215 0.2689414322376251
3279 0.9992367029190063 0.731056809425354
3280 0.9826205372810364 0.7310584783554077
3281 0.00029260604060254986 0.2692214846611023
3282 0.9025434255599976 0.7310580611228943
3283 0.0009397048270329832 0.26894211769104004
3284 0.001318082213401

...

16593 0.002315470715984702 0.2710733711719513
16594 0.28120556473731995 0.7229833602905273
16595 0.00029926747083663935 0.2689414322376251
16596 0.06776855140924454 0.2689426839351654
16597 0.9998639822006226 0.7310540080070496
16598 0.999135673046112 0.7310585975646973
16599 0.00018009341147262606 0.2689414322376251
16600 0.0030912889633327727 0.2689414322376251
16601 0.9704593420028688 0.7310564517974854
16602 0.18769969046115875 0.6958102583885193
16603 0.015758277848362926 0.26941901445388794
16604 0.9837226271629332 0.7309924960136414
16605 0.949753999710083 0.7310492992401123
16606 0.9929335117340088 0.7310365438461304
16607 0.99614018201828 0.7310156822204591
16608 0.9204552173614502 0.7310467958450317
16609 0.9558559656143188 0.7310226559638977
16610 0.001144678913988173 0.27140799164772034
16611 0.007813234813511372 0.2689415216445923
16612 0.029542418196797367 0.2689414322376251
16613 0.00014076061779633164 0.2689414918422699
16614 0.9981215596199036 0.7310585379600525
16615 

17375 0.0005786979454569519 0.2689414322376251
17376 0.0015907299239188433 0.2689414322376251
17377 0.9891575574874878 0.7310585975646973
17378 0.022691726684570312 0.26906758546829224
17379 0.0003942286421079189 0.26897022128105164
17380 0.01051255688071251 0.2689618170261383
17381 0.9978785514831544 0.3688700199127197
17382 0.00830416288226843 0.2689436972141266
17383 0.9985985159873962 0.7310546040534973
17384 0.9977158308029176 0.7294226884841919
17385 0.8668493032455444 0.7305545806884766
17386 0.9970014691352844 0.7310553789138794
17387 0.0009617813047952949 0.27118635177612305
17388 0.7411845326423645 0.2690317928791046
17389 0.9809072613716124 0.7305571436882019
17390 0.18054740130901334 0.270977795124054
17391 0.9745474457740784 0.7310574650764465
17392 0.0004470408603083343 0.27173617482185364
17393 0.6613134741783142 0.7300444841384888
17394 0.00046124981599859893 0.2689414322376251
17395 0.9910752773284912 0.7130420804023743
17396 0.991751492023468 0.7310130596160889
17397 

18129 0.0012241931399330495 0.26917141675949097
18130 0.9992374181747437 0.7310583591461182
18131 0.00048600471927784383 0.2690231502056122
18132 0.00014693578123115006 0.26899927854537964
18133 0.020889334380626682 0.2689491808414459
18134 0.9685298204421996 0.7310270071029663
18135 0.00013711386418435725 0.2689414322376251
18136 0.9790350198745728 0.730651319026947
18137 0.00020942732226103544 0.2689414322376251
18138 0.9921243786811828 0.7306739091873169
18139 0.9581019282341005 0.7308964133262634
18140 0.995335042476654 0.7310527563095093
18141 0.6222956776618958 0.7310564517974854
18142 0.038956962525844574 0.2698062360286713
18143 0.010477733798325062 0.7060791254043579
18144 0.997380554676056 0.7306814789772034
18145 0.93507581949234 0.2689976096153259
18146 0.05648249015212059 0.3007057905197144
18147 0.9988746047019958 0.7310583591461182
18148 0.14603544771671295 0.34944358468055725
18149 0.902945637702942 0.6707083582878113
18150 0.007430457975715399 0.2728002071380615
18151 

18951 0.9845529794692992 0.7310377359390259
18952 0.003392523154616356 0.2689414322376251
18953 0.0003838155826088041 0.2689414322376251
18954 0.9997162222862244 0.7310584783554077
18955 0.9979966282844543 0.7298892140388489
18956 0.9975225329399108 0.7310570478439331
18957 0.9951366782188416 0.2765830755233765
18958 0.022211385890841484 0.2689496278762817
18959 0.0007668875041417778 0.26923757791519165
18960 0.015930069610476494 0.2699355185031891
18961 0.9063832759857178 0.7310369610786438
18962 0.0010063617955893278 0.2689724862575531
18963 0.016921581700444218 0.2689422070980072
18964 0.055745698511600494 0.6213213205337524
18965 0.9848920106887816 0.7307684421539307
18966 0.9863914251327516 0.7310585379600525
18967 0.9902082681655884 0.7310572266578674
18968 0.9950454235076904 0.7308744788169861
18969 0.5107662677764893 0.26899102330207825
18970 0.9837669134140016 0.7310584783554077
18971 0.9795237183570862 0.7308829426765442
18972 0.9871066212654114 0.7308885455131531
18973 0.000

KeyboardInterrupt: 

In [28]:
file_path = "E:/lv_python/NLP/阿里天池小布助手/finetuned_results/"
filename2 = "new_result.tsv"
with open(file_path + filename2, "w") as fw:
        for idx, (score, score1)  in enumerate(zip(data["score"], data["score"])):
            new_score = (score + score1)/ 2
            fw.write(str(new_score))
            if idx == len(data) - 1:
                break
            fw.write("\n")

In [5]:
import os
os.listdir("E:/lv_python/NLP/阿里天池小布助手/finetuned_results/")

['2021-03-07_Linear_result.tsv',
 '2021-03-07_Linear_result.zip',
 '2021-03-07_result&oldtrain.zip',
 '2021-03-07_result.zip',
 'ChineseRobertaAttention_2021-03-02_result.zip',
 'ChineseRobertaAttention_2021-03-03_result.zip',
 'ChineseRobertaAttention_2021-03-06_result.zip',
 'ChineseRobertaLinear_2021-03-02_result.zip',
 'TextCNN_2021-03-11_result.tsv',
 'TextCNN_2021-03-11_result.zip']

In [6]:
os.path.exists("E:/lv_python/NLP/阿里天池小布助手/finetuned_results/2021-03-07_Linear_result.csv")

False

In [123]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from gensim.models import Word2Vec
import numpy as np

train_data = pd.read_csv("../tianchi_datasets/track3_round1_newtrain3.tsv", sep="\t", header=None,
                                 names=["sentence1", "sentence2", "labels"])
train_data["document"] = train_data["sentence1"].str.cat(train_data["sentence2"], sep = " [SEP] ")
test_data = pd.read_csv("../tianchi_datasets/track3_round1_testA.tsv", sep="\t", header=None,
                                names=["sentence1", "sentence2"])
test_data["document"] = test_data["sentence1"].str.cat(test_data["sentence2"], sep=" [SEP] ")

tokenizer = keras.preprocessing.text.Tokenizer(num_words = 7000, lower=False, filters = "", split = " ")
tokenizer.fit_on_texts(list(train_data["document"].values) + list(test_data["document"].values))
word_vocab = tokenizer.word_index

all_data = pd.concat([train_data["document"], test_data["document"]])
file_name = "./Word2vec_200.model"
model = Word2Vec([[word for word in document.split(" ")] for document in all_data.values],
                size = 400, window = 5, iter = 1, workers=12, seed = 2021, min_count = 2, sg = 1)
model.save(file_name)
model.wv.save_word2vec_format("./Word2Vec_200.bin", binary= True)

embedding_matrix = np.zeros((len(word_vocab), 400))
for word, i in word_vocab.items():
    embedding_vector = model.wv[word] if word in model else None
    if embedding_matrix is None:
        embedding_matrix[i-1] = embedding_vector
    else:
        unk_vec = np.random.random(400) * 0.5
        unk_vec = unk_vec - unk_vec.mean()
        embedding_matrix[i-1] = unk_vec
        
Word2Vec_model = gensim.models.KeyedVectors.load_word2vec_format('./Word2Vec_200.bin',binary=True)

d:\anaconda\envs\gluon1\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).


In [57]:
from tensorflow import keras

In [58]:
from gensim.models import Word2Vec

In [59]:
tokenizer = keras.preprocessing.text.Tokenizer(num_words = 7000, lower=False, filters = "", split = " ")

In [62]:
tokenizer.fit_on_texts(list(train_data["document"].values) + list(test_data["document"].values))

In [63]:
train_ = tokenizer.texts_to_sequences(train_data["document"].values)

In [64]:
test_ = tokenizer.texts_to_sequences(test_data["document"].values)


In [65]:
train_ = keras.preprocessing.sequence.pad_sequences(train_, maxlen = 120, padding = "post")
test_ = keras.preprocessing.sequence.pad_sequences(test_, maxlen = 120, padding = "post")

In [66]:
word_vocab = tokenizer.word_index

In [69]:
all_data = pd.concat([train_data["document"], test_data["document"]])
file_name = "./Word2vec_200.model"
model = Word2Vec([[word for word in document.split(" ")] for document in all_data.values],
                size = 400, window = 5, iter = 1, workers=12, seed = 2021, min_count = 2, sg = 1)

In [ ]:
model.save()

In [76]:
import numpy as np

In [84]:
embedding_matrix = np.zeros((len(word_vocab), 400))
for word, i in word_vocab.items():
    embedding_vector = model.wv[word] if word in model else None
    if embedding_matrix is None:
        embedding_matrix[i-1] = embedding_vector
    else:
        unk_vec = np.random.random(400) * 0.5
        unk_vec = unk_vec - unk_vec.mean()
        embedding_matrix[i-1] = unk_vec


d:\anaconda\envs\gluon1\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [86]:
len(embedding_matrix)

20601

In [87]:
import torch.nn as nn

In [95]:
import torch

In [96]:
emb = nn.Embedding.from_pretrained(torch.from_numpy(embedding_matrix))

In [99]:
import gensim

In [116]:
model.wv.save_word2vec_format("./Word2Vec_200.bin", binary= True)

In [118]:
Glove_model = gensim.models.KeyedVectors.load_word2vec_format('./Word2Vec_200.bin',binary=True)

In [130]:
import os
os.path.join("aa", "11")

'aa\\11'

In [146]:
emb(torch.LongTensor([[1]]))

tensor([[[ 8.6578e-02,  3.0881e-02,  5.4454e-02, -7.3786e-02, -2.2100e-01,
          -1.8517e-01, -9.1496e-02, -4.8742e-02,  6.2208e-02,  1.9237e-01,
          -2.3942e-01, -1.9800e-01, -2.1278e-01, -1.2114e-01, -1.4666e-01,
          -2.4913e-01,  3.7598e-02,  1.5293e-01, -1.1670e-03,  1.2145e-01,
          -1.8280e-02, -3.7732e-02, -9.9101e-02, -1.0589e-01,  1.8027e-01,
           2.2108e-01,  2.3163e-01,  2.3165e-01, -1.8647e-01,  8.5880e-02,
          -3.3081e-05, -1.1212e-01,  7.7813e-03,  2.1350e-01,  2.2863e-01,
          -2.0667e-02,  1.5393e-01, -3.8942e-02,  2.3829e-01,  1.8068e-01,
          -1.6188e-02, -6.8675e-03,  9.3522e-02,  5.4541e-02,  1.3982e-01,
           2.2968e-01,  7.4804e-02,  1.4016e-01,  7.9399e-02, -1.6915e-01,
           2.2485e-01,  1.0217e-01, -8.6826e-02,  8.4854e-02,  1.2325e-01,
          -2.2097e-01, -1.6086e-01,  8.9137e-02, -1.0869e-01, -2.1970e-01,
          -2.2921e-01, -1.6565e-01, -7.2503e-02,  8.4164e-03,  1.2088e-01,
           1.2985e-01,  9

In [145]:
embedding_matrix[0]

array([ 0.21711851, -0.11453386,  0.20077436, -0.05951488, -0.01232518,
        0.03963866,  0.24062929,  0.03123681, -0.04824853,  0.17596446,
        0.23107955,  0.09757179, -0.11815269, -0.07931834,  0.03433276,
        0.07799488, -0.04514416, -0.04940597,  0.19927388,  0.00592177,
       -0.16083231,  0.0315973 ,  0.00370703,  0.04645856, -0.07964959,
       -0.22261941,  0.18239956,  0.19892876, -0.02461764, -0.13521182,
       -0.02068985,  0.08519185, -0.2274013 , -0.11952821,  0.243154  ,
        0.20452446,  0.1480278 , -0.17845782,  0.13793747,  0.13007339,
       -0.24536355,  0.22856525, -0.02006326, -0.16872926, -0.17575469,
        0.11197394,  0.19310794, -0.11045123,  0.08552285,  0.00681663,
        0.24139888,  0.04866541, -0.07261371,  0.1455036 ,  0.15419056,
        0.00046861,  0.06077297, -0.01067   ,  0.15642028,  0.23849347,
       -0.2077974 , -0.07605521,  0.20200524, -0.21107796, -0.24078328,
        0.05012004, -0.04569194, -0.03846197,  0.24694546,  0.14

In [143]:
emb(torch.LongTensor([[1]]))

torch.Size([1, 1, 400])

In [133]:
len(word_vocab)

20601

In [131]:
word_vocab

{'[SEP]': 1,
 '12': 2,
 '29': 3,
 '19': 4,
 '23': 5,
 '16': 6,
 '11': 7,
 '10': 8,
 '9': 9,
 '32': 10,
 '39': 11,
 '161': 12,
 '126': 13,
 '48': 14,
 '243': 15,
 '43': 16,
 '106': 17,
 '5': 18,
 '6': 19,
 '72': 20,
 '66': 21,
 '8': 22,
 '13': 23,
 '283': 24,
 '282': 25,
 '459': 26,
 '28': 27,
 '370': 28,
 '317': 29,
 '62': 30,
 '76': 31,
 '247': 32,
 '140': 33,
 '276': 34,
 '50': 35,
 '46': 36,
 '169': 37,
 '80': 38,
 '300': 39,
 '47': 40,
 '202': 41,
 '59': 42,
 '217': 43,
 '431': 44,
 '45': 45,
 '127': 46,
 '134': 47,
 '426': 48,
 '536': 49,
 '67': 50,
 '447': 51,
 '230': 52,
 '24': 53,
 '133': 54,
 '522': 55,
 '52': 56,
 '533': 57,
 '432': 58,
 '227': 59,
 '251': 60,
 '226': 61,
 '253': 62,
 '453': 63,
 '168': 64,
 '358': 65,
 '629': 66,
 '25': 67,
 '272': 68,
 '176': 69,
 '267': 70,
 '102': 71,
 '55': 72,
 '462': 73,
 '153': 74,
 '79': 75,
 '421': 76,
 '173': 77,
 '130': 78,
 '334': 79,
 '347': 80,
 '101': 81,
 '524': 82,
 '525': 83,
 '335': 84,
 '44': 85,
 '360': 86,
 '200': 87,
 

In [148]:
emb1 = nn.Embedding.from_pretrained(torch.FloatTensor([[1, 2, 3], [4, 5, 6]]))

In [152]:
emb1(torch.LongTensor([[0]]))

tensor([[[1., 2., 3.]]])

In [155]:
emb = nn.Embedding.from_pretrained(torch.from_numpy(embedding_matrix))

In [157]:
emb(torch.LongTensor([[0]]))

tensor([[[ 0.2171, -0.1145,  0.2008, -0.0595, -0.0123,  0.0396,  0.2406,
           0.0312, -0.0482,  0.1760,  0.2311,  0.0976, -0.1182, -0.0793,
           0.0343,  0.0780, -0.0451, -0.0494,  0.1993,  0.0059, -0.1608,
           0.0316,  0.0037,  0.0465, -0.0796, -0.2226,  0.1824,  0.1989,
          -0.0246, -0.1352, -0.0207,  0.0852, -0.2274, -0.1195,  0.2432,
           0.2045,  0.1480, -0.1785,  0.1379,  0.1301, -0.2454,  0.2286,
          -0.0201, -0.1687, -0.1758,  0.1120,  0.1931, -0.1105,  0.0855,
           0.0068,  0.2414,  0.0487, -0.0726,  0.1455,  0.1542,  0.0005,
           0.0608, -0.0107,  0.1564,  0.2385, -0.2078, -0.0761,  0.2020,
          -0.2111, -0.2408,  0.0501, -0.0457, -0.0385,  0.2469,  0.1462,
          -0.1372, -0.0140, -0.1053, -0.1936,  0.0068, -0.2152, -0.0693,
           0.0827, -0.0820, -0.2165, -0.0807,  0.1375, -0.1709, -0.0499,
           0.1620,  0.1435,  0.1577,  0.2256,  0.0677, -0.2259, -0.0060,
           0.0784,  0.1071, -0.2328, -0.0948, -0.04

In [1]:
import time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round(elapsed))
    return str(datetime.timedelta(seconds = elapsed_rounded))

In [2]:
format_time(1000)

'0:16:40'

In [69]:
from Data_generator_vocab import Corpus

HYPERS = {
        "BATCH_SIZE":4,
        "LR": 5e-5,
        "EPOCHS": 7,
    }
seed_val = 2021
corpus = Corpus(HYPERS["BATCH_SIZE"], seed_val)
class_num = 2
maxlen = corpus.max_len
max_features = len(corpus.vocab)
embedding_dims = 400
dropout_rate = 0.2
feature_size = 1024
kernel_sizes = [2, 3, 4, 5]


********************************Data_precessing*********************************
## dataset size is 121105
*******************************Load  dataset ....*******************************
## Load Datasets Consume 0:00:10 s ###
************************All Train and Test Data loaded !************************


In [70]:
len(corpus.v2i)

20601

In [71]:
max_features

20601

In [177]:
from imp import reload
import TEXTCNN_pytorch
reload(TEXTCNN_pytorch)

<module 'TEXTCNN_pytorch' from 'E:\\lv_python\\NLP\\阿里天池小布助手\\TEXTCNN\\TEXTCNN_pytorch.py'>

In [190]:
from TEXTCNN_pytorch import TEXTCNN

model = TEXTCNN(maxlen=maxlen,
                    max_features=max_features,
                    embedding_dims=embedding_dims,
                    class_num=class_num,
                    kernel_sizes=kernel_sizes,
                    dropout_rate=dropout_rate,
                    feature_size=feature_size,
                    vocab = corpus.v2i)

In [192]:
model.embedding.weight

Parameter containing:
tensor([[-0.0920,  0.1511, -0.1715,  ...,  0.0840,  0.0454,  0.0940],
        [ 0.0316,  0.0616, -0.2723,  ...,  0.0495,  0.1081, -0.1547],
        [-0.0801, -0.0385, -0.2032,  ..., -0.0295,  0.1278, -0.0822],
        ...,
        [-0.0310,  0.1338, -0.0175,  ..., -0.0405, -0.2103, -0.0706],
        [-0.2232,  0.0664,  0.1908,  ...,  0.1807, -0.2016,  0.0979],
        [-0.2362, -0.2368, -0.0489,  ...,  0.1335, -0.1479,  0.0074]],
       requires_grad=True)

In [111]:
import gensim
Word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('./Word2Vec/Word2Vec_400.bin', binary=True)

In [115]:
import numpy as np

In [117]:
word_vocab = corpus.v2i

In [151]:
x = list(range(1, 10))
y = list(range(5, 10))
for data in x:
    z = data if data in y else None
    if z is not None:
        print(z)
    else:
        print("gg")

gg
gg
gg
gg
5
6
7
8
9


In [173]:
embedding_matrix

array([[-0.0919719 ,  0.15109034, -0.17147592, ...,  0.08398658,
         0.04537553,  0.09403314],
       [ 0.03163421,  0.0616281 , -0.2723169 , ...,  0.04948602,
         0.10811686, -0.15474498],
       [-0.08008043, -0.0385064 , -0.20319606, ..., -0.02954379,
         0.12779486, -0.08224422],
       ...,
       [ 0.14239134,  0.1760623 , -0.08992541, ...,  0.1655424 ,
         0.176705  ,  0.14613433],
       [ 0.0143168 , -0.19462781, -0.17690691, ..., -0.10978252,
        -0.11064705, -0.00087031],
       [-0.20442522, -0.05232566,  0.21068117, ..., -0.15534113,
         0.21882315,  0.08652651]])

In [164]:
cnt

20601

In [154]:
word = "245"
embedding_vector = Word2vec_model.wv[word] if word in Word2vec_model else None

In [156]:
if embedding_vector is not None:
    print(1)
else:
    print(2)

2


In [147]:
embedding_vector is not None

False

In [127]:
word in Word2vec_model

True

In [11]:
import os
os.mkdir("./Word2Vec")

In [74]:
train_loader, valid_loader, test_loader = corpus.get_loaders()

In [75]:
for batch in train_loader:
    inputs = batch[:-1]
    break

In [76]:
inputs

[tensor([[   1,   14,   63,    3,   15,  664,    0,    1,   14,  331,  664,    5,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0],
         [  94,   31, 1538,  135,  135,  164,    0, 1593,  165,   17,  177,    4,
            15,  562,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,

In [80]:
model(*inputs)

tensor([[0.4549, 0.5451],
        [0.4578, 0.5422],
        [0.4651, 0.5349],
        [0.4621, 0.5379]], grad_fn=<SoftmaxBackward>)

In [45]:
import torch.nn as nn

In [97]:
emb = nn.Embedding(num_embeddings=max_features, embedding_dim=embedding_dims)

In [98]:
emb.weight.requires_grad

True

In [61]:
model.embedding.weight.dtype

torch.float32

In [130]:
model.embedding.weight

Parameter containing:
tensor([[-0.0920,  0.1511, -0.1715,  ...,  0.0840,  0.0454,  0.0940],
        [ 0.0316,  0.0616, -0.2723,  ...,  0.0495,  0.1081, -0.1547],
        [-0.0801, -0.0385, -0.2032,  ..., -0.0295,  0.1278, -0.0822],
        ...,
        [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
        [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
        [    nan,     nan,     nan,  ...,     nan,     nan,     nan]],
       requires_grad=True)

In [109]:
model.embedding_matrix

array([[-0.12898413,  0.00944187, -0.20704381, ...,  0.15879674,
         0.25146197,  0.17583833],
       [-0.24980616,  0.14945459, -0.16667096, ...,  0.05273612,
         0.19400932, -0.12150745],
       [ 0.00146409, -0.05594294,  0.05401677, ..., -0.1377127 ,
         0.06735486, -0.04046536],
       ...,
       [ 0.08236733,  0.24451374,  0.23731845, ..., -0.17505785,
         0.16749684, -0.14093859],
       [ 0.15026279, -0.1254758 , -0.04966982, ..., -0.07099763,
         0.04706879, -0.1423259 ],
       [-0.2133781 ,  0.22889107, -0.22282936, ..., -0.24597387,
         0.2347286 , -0.17734471]])

In [44]:
print(model)

TEXTCNN(
  (embedding): Embedding(20601, 400)
  (conv1ds): ModuleList(
    (0): Conv1d(400, 1024, kernel_size=(2,), stride=(1,))
    (1): Conv1d(400, 1024, kernel_size=(3,), stride=(1,))
    (2): Conv1d(400, 1024, kernel_size=(4,), stride=(1,))
    (3): Conv1d(400, 1024, kernel_size=(5,), stride=(1,))
  )
  (relu): ReLU()
  (softmax): Softmax()
  (avgpools): ModuleList(
    (0): MaxPool1d(kernel_size=90, stride=90, padding=0, dilation=1, ceil_mode=False)
    (1): MaxPool1d(kernel_size=89, stride=89, padding=0, dilation=1, ceil_mode=False)
    (2): MaxPool1d(kernel_size=88, stride=88, padding=0, dilation=1, ceil_mode=False)
    (3): MaxPool1d(kernel_size=87, stride=87, padding=0, dilation=1, ceil_mode=False)
  )
  (dropout): SpatialDropout()
  (linear): Linear(in_features=4096, out_features=512, bias=True)
  (classifier): Linear(in_features=512, out_features=2, bias=True)
)


In [64]:
import torch

In [194]:
import pandas as pd

In [202]:
data1 = pd.read_csv("../finetuned_results/2021-03-07_Linear_result.tsv", sep = ",", header = None, names = ["score"])
data2 = pd.read_csv("./finetuned_results/TextCNN_2021-03-13_result.tsv", sep = ",", header = None, names = ["score"])

In [199]:
file_path = "../finetuned_results"
filename2 = "merge_result.tsv"
import os
os.path.join(file_path, filename2)

'../finetuned_results\\merge_result.tsv'

In [204]:
file_path = "../finetuned_results"
filename2 = "merge_result.tsv"
with open(os.path.join(file_path, filename2), "w") as fw:
    for idx, (score1, score2) in enumerate(zip(data1["score"], data2["score"])):
        score = (score1 + score2)/2
        fw.write(str(score))
        if idx == len(data1) - 1:
            break
        fw.write("\n")